# 2D Spatial Cross-Correlation

In [ ]:
########################################################################################
# Author: Justin Clancy, University of Melbourne, 2020                                 #
# - Pattern Matching -                                                                 #
# Measuring Spatial Cross-Correlation for 2D Images                                    #
########################################################################################

## Import Modules

In [ ]:
# Load necessary modules
import numpy as np
from PIL import Image, ImageDraw
from numpy import loadtxt
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from timeit import default_timer as timer

## Read Image Files

In [ ]:
# Function to read in image and convert it to greyscale
def g_imarray(file):
    # Read image and ('L') converts it to greyscale
    im = np.array(Image.open(file).convert('L'))
    return im



#----Not really needed, just a more in depth version of above----# 
    
def greyscale(colour):
    r, g, b = colour[:,:,0], colour[:,:,1], colour[:,:,2]
    grey = 0.2989*r + 0.5870*g + 0.1140*b
    return grey
#----------------------------------------------------------------#



#-----Test-----#


# Import files, convert to greyscale and turn into a 2D array
gs_p = g_imarray("/Users/justi/Desktop/wallypuzzle_rocket_man.png")
gs_t = g_imarray("/Users/justi/Desktop/wallypuzzle_png.png")

# Import images without greyscale conversion for comparison
template = np.array(Image.open("/Users/justi/Desktop/wallypuzzle_png.png"))
pattern = np.array(Image.open("/Users/justi/Desktop/wallypuzzle_rocket_man.png"))

## 2D Zero-Padding

In [ ]:
# Define a function to pad a 2D array
def spatial_pad(array, padding=1):
    # Write out the array dimensions
    x,y = array.shape
    # Create an empy 2D array of at least 2 values larger in x and y
    # value multiplied by an input depending on how much padding
    # is required
    padded = np.zeros((x+2*padding, y+2*padding),dtype=array.dtype)
    # Fill the original dimensions in the center of the padded array
    # with the original array values
    padded[padding:-padding, padding:-padding] = array
    # Return new array
    return padded

## 2D best lag

In [ ]:
# Redefine a function for the best lag to search a 2D array. This is 
# pretty straight forward
def b_lag(scores,pattern):
    # Check for the best lag as a position in x and y separetly
    y,x  = np.unravel_index(np.argmax(scores),scores.shape)
    # Depending on what the pattern is, it is better generally to 
    # consider shifting the best lag to correspond with the center of 
    # pattern rather than the top left corner by default
    xmid = x + pattern.shape[1] / 2
    ymid = y + pattern.shape[0] / 2
    return xmid,ymid

## 2D Normalized Spatial Cross-Correlation Score

In [ ]:
# Spectral Cross-Correlation function normalized with array energy

# A separate array energy function was not really necessary so 
# the functions were combined
def norm_corr_2d(pattern,template):
    # Follow calculations as made in the 1D signal cross correlation
    correlation = np.sum(pattern*template)
    energy = np.sqrt((np.sum(template**2)))*np.sqrt(np.sum(pattern**2))
    # Return normalized score
    return correlation/energy

## Submission Question

In [ ]:
# A functoin to take an input pattern and template and locate the
# pattern in the template, placing a black circle on the location
def wheres_rocketman(pattern_file, template_file):
    
    # Read files as greyscale images with integer values
    pattern = g_imarray(pattern_file)
    pattern = np.array(pattern,dtype='int')
    template = g_imarray(template_file)
    
    # Pad the template in both dimensions by a set amount
    template = spatial_pad(template,100)
    template = np.array(template,dtype='int')

    # Read template as coloured image for viewing final result
    template_colour = np.array(Image.open(template_file))
    
    # Write out pattern and template dimensions
    temp_h,temp_w = template.shape
    pat_h,pat_w = pattern.shape
    
    # Create an empty array to be filled with the scores
    score_arr = np.zeros((temp_h-pat_h,temp_w-pat_w))
    
    # Iterate the pattern row by row across the template
    for y in range(0,temp_h-pat_h):
        for x in range(0,temp_w-pat_w):
            # Define current template slice (m)
            temp_slice = template[y:y+pat_h,x:x+pat_w]
            # Calculate score and save in the score array
            score_arr[y,x] = norm_corr_2d(pattern,temp_slice)
            
    # Obtain best lag location 
    blagx = b_lag(score_arr,pattern)[0]-100
    blagy = b_lag(score_arr,pattern)[1]-100

    # Create a black marking circle at the location of best lag
    rocket_man = Circle((blagx,blagy),radius=20,color='black')

    
    # Plot the result on the original coloured image
    fig, ax = plt.subplots(1,figsize=(38,12))
    ax.imshow(template_colour)
    ax.add_patch(rocket_man)
    plt.show(fig)

    print("Rocket man at:",(blagx,blagy))
    return blagx,blagy

## Run the function

In [ ]:
# Find the 'Rocket man' in the Where's Wally puzzle and obtain runtime
start = timer()
wheres_rocketman("/Users/justi/Desktop/wallypuzzle_rocket_man.png","/Users/justi/Desktop/wallypuzzle_png.png")
end = timer()
print("Runtime:", end-start,"s") # Runtime in seconds